<a href="https://colab.research.google.com/github/joren015/MercariProject/blob/main/CSCI5523_Mercari_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download dataset from Kaggle using Kaggle API
Create a Kaggle API key with your Kaggle account and upload the json to the Google Colab root directory.

In [1]:
%%shell
mkdir /root/.kaggle
chmod 600 /root/.kaggle/kaggle.json
mv ./kaggle.json /root/.kaggle/kaggle.json
kaggle competitions download -c mercari-price-suggestion-challenge
7za e train.tsv.7z
7za e test.tsv.7z

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
 93% 69.0M/74.3M [00:00<00:00, 106MB/s] 
100% 74.3M/74.3M [00:00<00:00, 137MB/s]
  0% 0.00/170k [00:00<?, ?B/s]
100% 170k/170k [00:00<00:00, 137MB/s]
 65% 22.0M/34.0M [00:00<00:00, 56.8MB/s]
100% 34.0M/34.0M [00:00<00:00, 113MB/s] 
 96% 282M/294M [00:02<00:00, 144MB/s]
100% 294M/294M [00:02<00:00, 127MB/s]
  0% 0.00/7.77M [00:00<?, ?B/s]
100% 7.77M/7.77M [00:00<00:00, 71.4MB/s]

7-Zip (a) [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 77912192 bytes (75 MiB)

Extracting archive: train.tsv.7z
--
Path = train.tsv.7z
Type = 7z
Physical Size = 77912192
Headers Size = 122
Method = LZMA2:24
Solid = -
Blocks = 1

  0%      3% - train.tsv                  7% - tr

### Import packages

In [2]:
import pandas as pd

### Define global variables

In [3]:
root_dir = "."

### Load train and test datasets

In [4]:
df_train = pd.read_csv("train.tsv", sep='\t')
df_test = pd.read_csv("test.tsv", sep='\t')

### Functions

In [5]:
def describe_df(df):
  for col in df.columns:
    print(col)
    print(df[col].describe())
    print("null/nan count: {}".format(len(df) - len(df[col].dropna())))
    print("\n")

### Describe datasets

##### Train

In [6]:
describe_df(df_train)

train_id
count    1.482535e+06
mean     7.412670e+05
std      4.279711e+05
min      0.000000e+00
25%      3.706335e+05
50%      7.412670e+05
75%      1.111900e+06
max      1.482534e+06
Name: train_id, dtype: float64
null/nan count: 0


name
count     1482535
unique    1225273
top        Bundle
freq         2232
Name: name, dtype: object
null/nan count: 0


item_condition_id
count    1.482535e+06
mean     1.907380e+00
std      9.031586e-01
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      3.000000e+00
max      5.000000e+00
Name: item_condition_id, dtype: float64
null/nan count: 0


category_name
count                                            1476208
unique                                              1287
top       Women/Athletic Apparel/Pants, Tights, Leggings
freq                                               60177
Name: category_name, dtype: object
null/nan count: 6327


brand_name
count     849853
unique      4809
top         PINK
freq       54088
Name: br

##### Test

In [7]:
describe_df(df_test)

test_id
count    693359.000000
mean     346679.000000
std      200155.646984
min           0.000000
25%      173339.500000
50%      346679.000000
75%      520018.500000
max      693358.000000
Name: test_id, dtype: float64
null/nan count: 0


name
count     693359
unique    601117
top       Bundle
freq        1111
Name: name, dtype: object
null/nan count: 0


item_condition_id
count    693359.000000
mean          1.906102
std           0.903378
min           1.000000
25%           1.000000
50%           2.000000
75%           3.000000
max           5.000000
Name: item_condition_id, dtype: float64
null/nan count: 0


category_name
count                                             690301
unique                                              1223
top       Women/Athletic Apparel/Pants, Tights, Leggings
freq                                               27900
Name: category_name, dtype: object
null/nan count: 3058


brand_name
count     397834
unique      3900
top         Nike
freq       2523

##### Unique sub categories

In [8]:
df = pd.concat([df_train, df_test])
unique_categories = set(df["category_name"].apply(lambda x: str(x)).unique())
sub_categories = set()
for category in unique_categories:
  sub_categories = sub_categories.union(set(category.split('/')))

sub_categories = list(sub_categories)
sub_categories.sort()
print("Sub category count: {}".format(len(sub_categories)))
for c in sub_categories:
  print(c)

Sub category count: 962
100 Years or Older
50 To 75 Years
75 To 100 Years
A-Line
Above Knee, Mini
Accessories
Accessory
Aceo
Action Figure
Action Figures & Statues
Action, Adventure
Activity Centers & Entertainers
Advertisement
Afghan
Air Conditioners
Air Fresheners
Air Purifiers
Album
All Other Sports
Amigurumi
Amplifiers & Effects
Animal
Animals
Animation
Antique
Apparel
Apron
Area Rugs & Pads
Art
Art Doll
Artist Bears
Arts & Crafts
Arts & Photography
Artwork
Asymmetrical
Asymmetrical Hem
Athletic
Athletic Apparel
Athletic Training
Automotive
Automotive Enthusiast Merchandise
Baby
Baby & Child Care
Baby & Toddler Toys
Baby Gyms & Playmats
Baby Seats
Backpack
Backpack Style
Backpacks & Carriers
Backpacks, Bags & Briefcases
Badminton
Baggy, Loose
Bags & Cases
Bags and Purses
Baguette
Bakeware
Ballet
Band & Orchestra
Baseball
Baseball & Softball
Basic Supplies
Basket
Basketball
Baskets
Baskets & Bins
Bass Guitars
Bath
Bath & Body
Bath Linen Sets
Bath Rugs
Bathing & Skin Care
Bathing Acc

In [9]:
for item_description in df["item_description"][0:25]:
  print(item_description)
  print()

No description yet

This keyboard is in great condition and works like it came out of the box. All of the ports are tested and work perfectly. The lights are customizable via the Razer Synapse app on your PC.

Adorable top with a hint of lace and a key hole in the back! The pale pink is a 1X, and I also have a 3X available in white!

New with tags. Leather horses. Retail for [rm] each. Stand about a foot high. They are being sold as a pair. Any questions please ask. Free shipping. Just got out of storage

Complete with certificate of authenticity

Banana republic bottoms, Candies skirt with matching blazer,Amy Byers suit, Loft bottoms and cami top.

Size small but straps slightly shortened to fit xs, besides that, perfect condition

You get three pairs of Sophie cheer shorts size small and medium girls and two sports bra/boy shorts spandex matching sets in small and medium girls. All items total retail for [rm] in store and you can take him today for less than the price of one item at 